<a href="https://colab.research.google.com/github/chaitanyap1/Datascience-projects/blob/master/Tag_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
tags='/content/drive/My Drive/stacksample/Tags.csv'

In [0]:
import pandas as pd
tags=pd.read_csv(tags)

In [0]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [0]:
top10=tags['Tag'].value_counts()[:10].index.tolist()

In [0]:
top10

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [0]:
top10= tags[tags.Tag.isin(top10)]
print (top10.shape)
top10.head()

(826739, 2)


,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [0]:
top10['Id'].value_counts().head(10)

30798790    5
31085960    5
11648170    5
35318730    5
4009250     5
30289880    5
23267320    5
35283570    5
30991580    5
23484760    5
Name: Id, dtype: int64

In [0]:
top10.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [0]:
top10.shape

(826739, 2)

In [0]:
tag_ques=top10.groupby('Id',as_index=False)['Tag'].agg(lambda x:x.tolist())

In [0]:
tag_ques

,Id,Tag
0,260,[c#]
1,330,[c++]
2,650,[c#]
3,930,[c#]
4,1010,[c#]
...,...,...
706331,40143150,[javascript]
706332,40143190,[python]
706333,40143210,[php]
706334,40143340,[android]


In [0]:
copied_path1 = 'drive/My Drive/stacksample/Questions.csv'
ques = pd.read_csv(copied_path1,encoding="ISO-8859-1")

In [0]:
ques.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [0]:
import re 

def rem_html_tags(body):
    regex = re.compile('<.*?>')
    return re.sub(regex, '', body)

In [0]:
ques['Body'] = ques['Body'].apply(rem_html_tags)
ques.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,This is something I've pseudo-solved many time...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...


In [0]:
total=pd.merge(ques, tag_ques, on='Id')
print(total.shape)
total.head()

(706336, 8)


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,[c#]
1,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,[c++]
2,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,I would like the version property of my applic...,[c#]
3,930,245.0,2008-08-04T00:47:25Z,NaN,28,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,[c#]
4,1010,67.0,2008-08-04T03:59:42Z,NaN,14,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,[c#]


In [0]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence


from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model



In [0]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(total.Tag)
labels = multilabel_binarizer.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [0]:
train,test=train_test_split(total[:550000],test_size=0.25,random_state=24)

In [0]:
train.shape,test.shape

((412500, 8), (137500, 8))

In [0]:
X_train_t=train['Title']
X_train_b=train['Body']
y_train=multilabel_binarizer.transform(train['Tag'])
X_test_t=test['Title']
X_test_b=test['Body']
y_test=multilabel_binarizer.transform(test['Tag'])

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sent_lens_t=[]
for sent in train['Title']:
    sent_lens_t.append(len(word_tokenize(sent)))
max(sent_lens_t)

59

In [0]:
import numpy as np

In [0]:
np.quantile(sent_lens_t,0.97)

18.0

In [0]:
max_len_t = 18
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
sequences_train_t = tok.texts_to_sequences(X_train_t)

In [0]:
vocab_len_t=len(tok.index_word.keys())
vocab_len_t

68969

In [0]:
sequences_matrix_train_t = sequence.pad_sequences(sequences_train_t,maxlen=max_len_t)
sequences_matrix_train_t

array([[    0,     0,     0, ...,     1,   957,   197],
       [    0,     0,     0, ...,  9081,    45,   533],
       [    0,     0,     0, ...,   147,     8,   230],
       ...,
       [    0,     0,     0, ...,    10,    71,  2985],
       [    0,     0,     0, ...,     2,    18,    75],
       [    0,     0,     0, ..., 11009,   809,   267]], dtype=int32)

In [0]:
sequences_test_t = tok.texts_to_sequences(X_test_t)
sequences_matrix_test_t = sequence.pad_sequences(sequences_test_t,maxlen=max_len_t)

In [0]:
sequences_matrix_train_t.shape,sequences_matrix_test_t.shape,y_train.shape,y_test.shape

((412500, 18), (137500, 18), (412500, 10), (137500, 10))

In [0]:
sent_lens_b=[]
for sent in train['Body']:
    sent_lens_b.append(len(word_tokenize(sent)))
max(sent_lens_b)

KeyboardInterrupt: ignored

In [0]:
np.quantile(sent_lens_b,0.90)

432.0

In [0]:
max_len_b = 400
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_b)
sequences_train_b = tok.texts_to_sequences(X_train_b)

In [0]:
vocab_len_b =len(tok.index_word.keys())
vocab_len_b

1292018

In [0]:
sequences_matrix_train_b = sequence.pad_sequences(sequences_train_b,maxlen=max_len_b)
sequences_matrix_train_b

array([[   0,    0,    0, ...,   51, 2082,   91],
       [   0,    0,    0, ..., 1408,  203,  825],
       [   0,    0,    0, ...,   34,   51,   83],
       ...,
       [   0,    0,    0, ...,   20,   68,  687],
       [   0,    0,    0, ...,  187,   58,   10],
       [   0,    0,    0, ...,  194,  197,   10]], dtype=int32)

In [0]:
sequences_test_b = tok.texts_to_sequences(X_test_b)
sequences_matrix_test_b = sequence.pad_sequences(sequences_test_b,maxlen=max_len_b)

In [0]:
sequences_matrix_train_t.shape,sequences_matrix_train_b.shape,y_train.shape

((412500, 18), (412500, 400), (412500, 10))

In [0]:
sequences_matrix_test_t.shape,sequences_matrix_test_b.shape,y_test.shape

((137500, 18), (137500, 400), (137500, 10))

In [0]:
def RNN():
    # Title Only
    title_input = Input(name='title_input',shape=[max_len_t])
    title_Embed = Embedding(vocab_len_t+1,2000,input_length=max_len_t,mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(300)(title_Embed)
    # auxiliary output to tune GRU weights smoothly 
    auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)   
    
    # Body Only
    body_input = Input(name='body_input',shape=[max_len_b]) 
    body_Embed = Embedding(vocab_len_b+1,170,input_length=max_len_b,mask_zero=True,name='body_Embed')(body_input)
    gru_out_b = GRU(200)(body_Embed)
    
    # combined with GRU output
    com = concatenate([gru_out_t, gru_out_b])
    
    # now the combined data is being fed to dense layers
    dense1 = Dense(400,activation='relu')(com)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1) 
    dense2 = Dense(150,activation='relu')(bn)
    
    main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    
    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    return model

In [0]:
model = RNN()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_input (InputLayer)        (None, 18)           0                                            
__________________________________________________________________________________________________
body_input (InputLayer)         (None, 400)          0                                            
__________________________________________________________________________________________________
title_Embed (Embedding)         (None, 18, 2000)     137940000   title_input[0][0]                
__________________________________________________________________________________________________
body_Embed (Embedding)          (None, 400, 170)     219643230   body_input[0][0]                 
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam',loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [0]:
results=model.fit({'title_input': sequences_matrix_train_t, 'body_input': sequences_matrix_train_b},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=5, batch_size=800)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:421: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 137940000 elements. This may consume a large amount of memory.
  num_elements)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:421: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 219643230 elements. This may consume a large amount of memory.
  num_elements)



Train on 412500 samples, validate on 137500 samples
Epoch 1/5





412500/412500 [==============================] - 579s 1ms/step - loss: 2.3511 - main_output_loss: 1.0640 - aux_output_loss: 1.2871 - main_output_acc: 0.7206 - aux_output_acc: 0.6649 - val_loss: 1.8060 - val_main_output_loss: 0.7567 - val_aux_output_loss: 1.0494 - val_main_output_acc: 0.8135 - val_aux_output_acc: 0.7254
Epoch 2/5
412500/412500 [==============================] - 566s 1ms/step - loss: 1.6071 - main_output_loss: 0.6675 - aux_output_loss: 0.9397 - main_output_acc: 0.8408 - aux_output_acc: 0.7587 - val_loss: 1.7145 - val_main_output_loss: 0.6705 - val_aux_output_loss: 1.0440 - val_main_output_acc: 0.8349 - val_aux_output_acc: 0.7290
Epoch 3/5
412500/412500 [==============================] - 560s 1ms/step - loss: 1.4152 - main_output_loss: 0.5634 - aux_output_loss: 0.8518 - main_output_acc: 0.8639 - aux_output_acc: 0.7780 - val_loss: 1.7375 - val_main_output_loss: 0.6655 - val_aux_output_loss: 1.0720 - val_ma

In [0]:
(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},verbose=1)

137500/137500 [==============================] - 1024s 7ms/step


In [0]:
from sklearn.metrics import classification_report,f1_score

In [0]:
print(f1_score(y_test,predicted_main>.55,average='samples'))

0.8315294891774891


In [0]:
print(classification_report(y_test,predicted_main>.55))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94     17054
           1       0.93      0.79      0.86     20681
           2       0.86      0.86      0.86      9700
           3       0.67      0.56      0.61     11304
           4       0.96      0.90      0.93      8897
           5       0.92      0.80      0.86     22472
           6       0.82      0.70      0.76     22938
           7       0.81      0.82      0.82     16150
           8       0.92      0.88      0.90     19659
           9       0.98      0.91      0.94     11576

   micro avg       0.89      0.81      0.85    160431
   macro avg       0.88      0.82      0.85    160431
weighted avg       0.89      0.81      0.85    160431
 samples avg       0.84      0.84      0.83    160431



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
test.iloc[24]

Id                                                       16470700
OwnerUserId                                           2.36783e+06
CreationDate                                 2013-05-09T20:39:16Z
ClosedDate                                                    NaN
Score                                                           1
Title           NetworkOnMainThreadException- Have tried makin...
Body            I've been trying to get this to work for a whi...
Tag                                               [java, android]
Name: 250148, dtype: object